In [2]:
import pandas as pd
import numpy as np
from collections import Counter
df=pd.read_csv('amazon_baby.csv')
pd.set_option('display.max_columns',100)

In [3]:
df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


# Fill the Nan value 

In [4]:
df['review']=df['review'].fillna('k')

In [5]:
import string
def punct_strip(word):
    for i in string.punctuation:
         word = word.replace(i, '')
    return word

# Word count 


In [6]:
df['word_count']=df['review'].apply(lambda x: Counter(punct_strip(x).lower().split()) if type(x)==str else 'k')

In [7]:
df.head()

,name,review,rating,word_count
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,"{'these': 1, 'flannel': 1, 'wipes': 3, 'are': ..."
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,"{'it': 3, 'came': 1, 'early': 1, 'and': 3, 'wa..."
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,"{'very': 1, 'soft': 1, 'and': 2, 'comfortable'..."
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,"{'this': 4, 'is': 4, 'a': 2, 'product': 2, 'we..."
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,"{'all': 2, 'of': 1, 'my': 1, 'kids': 2, 'have'..."


# Ignore 3 star rating and create sentiment column/ 

In [8]:
df=df[df['rating'] != 3]
df['sentiment']=df['rating'].apply(lambda x: 1 if x>=4 else 0)

In [9]:
# df['awesome']=df['word_count'].apply(lambda x : x['awesome'])
df=df.assign(awesome=df['word_count'].apply(lambda x:x['awesome']),
             great=df['word_count'].apply(lambda x: x['great']),
             fantastic=df['word_count'].apply(lambda x: x['fantastic']),
             amazing=df['word_count'].apply(lambda x: x['amazing']),
             love=df['word_count'].apply(lambda x: x['love']),
             horrible=df['word_count'].apply(lambda x: x['horrible']),
             bad=df['word_count'].apply(lambda x: x['bad']),
             terrible=df['word_count'].apply(lambda x: x['terrible']),
             awful=df['word_count'].apply(lambda x: x['awful']),
             wow=df['word_count'].apply(lambda x: x['wow']),
             hate=df['word_count'].apply(lambda x: x['hate']),
            )

In [10]:
for i in df.columns[5:16]:
    print("{0} ==".format(i),df[i].sum())
#     print(df[i].value_counts().sum())
# df.columns[4:16]
df.head()

awesome == 3735
great == 54398
fantastic == 1604
amazing == 2539
love == 41515
horrible == 1085
bad == 4044
terrible == 1101
awful == 657
wow == 380
hate == 1090


,name,review,rating,word_count,sentiment,awesome,great,fantastic,amazing,love,horrible,bad,terrible,awful,wow,hate
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,"{'it': 3, 'came': 1, 'early': 1, 'and': 3, 'wa...",1,0,0,0,0,1,0,0,0,0,0,0
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,"{'very': 1, 'soft': 1, 'and': 2, 'comfortable'...",1,0,0,0,0,0,0,0,0,0,0,0
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,"{'this': 4, 'is': 4, 'a': 2, 'product': 2, 'we...",1,0,0,0,0,2,0,0,0,0,0,0
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,"{'all': 2, 'of': 1, 'my': 1, 'kids': 2, 'have'...",1,0,1,0,0,0,0,0,0,0,0,0
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,"{'when': 2, 'the': 6, 'binky': 3, 'fairy': 3, ...",1,0,1,0,0,0,0,0,0,0,0,0


In [11]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

# Logistic regresion classifiar 

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

##  create logistic regression model 

In [13]:
x_train,x_test,y_train,y_test=train_test_split(df[selected_words],df['sentiment'],test_size=0.2,random_state=0)
logreg = LogisticRegression()
logreg.fit(x_train,y_train)

predictions=logreg.predict(x_test)

## measure accuracy of the model

In [14]:
score = logreg.score(x_test,y_test)
print(score*100)

84.51320799976013


## Confusion matrix


In [15]:
cm=metrics.confusion_matrix(y_test,predictions)
print(cm)

[[  344  5018]
 [  147 27842]]


In [16]:
s=logreg.coef_
print("Name   Coefficient")
print("------------------")
l=[]
for i,j in zip(df.columns[5:16],range(11)):
#     print("{0} ==".format(i),s[0][j])
    l.append([i,s[0][j]])
    
l=sorted(l,key=lambda x:x[1],reverse=True)
for i in l:
    print("{} == {}".format(i[0],i[1]))

Name   Coefficient
------------------
love == 1.3873337184967776
awesome == 1.178728071501063
amazing == 1.0337865051267605
fantastic == 0.8994186232550317
great == 0.8653686176632086
wow == -0.08072957229455362
bad == -0.9870445047695329
hate == -1.4338681601515761
awful == -2.0741767739460673
terrible == -2.197422975267511
horrible == -2.2815916861325114


In [1]:
diaper_champ_reviews = df[df['name']=="Baby Trend Diaper Champ" ]
print(list(diaper_champ_reviews["sentiment"]))
a=diaper_champ_reviews['sentiment']
diaper_champ_reviews=diaper_champ_reviews[selected_words]
predictions=logreg.predict(diaper_champ_reviews)

NameError: name 'df' is not defined

In [61]:
accuracy_sc=metrics.accuracy_score(a,predictions)
print(accuracy_sc)
cm=metrics.confusion_matrix(a,predictions)
print(cm)

0.8422818791946308
[[  4  46]
 [  1 247]]


In [19]:
diaper_champ_reviews1=diaper_champ_reviews[selected_words]
predictions=logreg.predict(diaper_champ_reviews1)
predictions

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [20]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=False)
X = v.fit_transform(df["word_count"][1])
X
df['word_count'][1]

Counter({'it': 3,
         'came': 1,
         'early': 1,
         'and': 3,
         'was': 1,
         'not': 2,
         'disappointed': 1,
         'i': 1,
         'love': 1,
         'planet': 1,
         'wise': 1,
         'bags': 1,
         'now': 1,
         'my': 2,
         'wipe': 1,
         'holder': 1,
         'keps': 1,
         'osocozy': 1,
         'wipes': 1,
         'moist': 1,
         'does': 1,
         'leak': 1,
         'highly': 1,
         'recommend': 1})